In [1]:
import numpy as np
import pickle

with open('cache/traintest.pkl', 'rb') as f:
    df_train, df_test, train_y = pickle.load(f)
    
with open('cache/traintest_decomposition2_50.pkl', 'rb') as f:
    out50_train, out50_test = pickle.load(f)
        
with open('cache/non_zero_stats.pkl', 'rb') as f:
    agg_train, agg_test = pickle.load(f)        

In [2]:
from sklearn.feature_selection import VarianceThreshold    
import pandas as pd
from sklearn import preprocessing, model_selection, metrics
import lightgbm as lgb
import xgboost as xgb


In [3]:
df_train.shape, out50_train.shape, agg_train.shape

((4459, 4735), (4459, 100), (4459, 18))

In [4]:


def low_variance(df_train):
    s = VarianceThreshold(threshold=.0)
    s.fit(df_train)
    idxs = s.get_support()
    to_remove = [c for c, x in zip(df_train.columns, idxs) if not x]
    return to_remove

def nunique(df_train):
    _, index = np.unique(df_train.values, return_index=True, axis=1)
    return [c for idx, c in enumerate(df_train.columns) if idx not in index]

def suspicious(df_train):
    # drop zero fraction
    thr = 0.98
    to_ignore = []
    for column in df_train.columns:
        counts = df_train[column].value_counts()
        value_fraction = counts[0] / len(df_train)
        if value_fraction >= thr:
            to_ignore.append(column)
    return to_ignore



In [5]:
to_ignore = set()
to_ignore |= set(low_variance(df_train))
to_ignore |= set(nunique(df_train))
to_ignore |= set(suspicious(df_train))

In [6]:
len(to_ignore)

2614

In [7]:
features = [c for c in df_train if c not in to_ignore]

In [8]:
len(features)

2121

In [9]:
#train_X = pd.concat([df_train[features], out50_train, agg_train], axis=1)
train_X = pd.concat([out50_train, agg_train], axis=1)
train_X = train_X.replace([np.inf, -np.inf], np.nan)
train_X = train_X.fillna(0)

#test_X = pd.concat([df_test[features], out100_test, agg_test], axis=1)
test_X = pd.concat([out50_test, agg_test], axis=1)
test_X = test_X.replace([np.inf, -np.inf], np.nan)
test_X = test_X.fillna(0)


In [10]:
train_X.shape

(4459, 118)

In [20]:

def run_cv2(train_X, train_y):    
    params = {
        'booster': 'gbtree',
        'objective': 'reg:linear', 
        'learning_rate': 0.01,
        'max_depth': 3, 
        #'min_child_weight': 1,
        #'subsample': 0.1,
        #'colsample_bytree': 0.1,
        #'colsample_bylevel': 0.1,
        #'reg_lambda': 1,
        #'reg_alpha': 1,
        'base_score': np.mean(train_y)
    }    
    
    xgtrain = xgb.DMatrix(train_X, label=train_y)
    hist = xgb.cv(params, 
                  xgtrain, 
                  10000, 
                  nfold=5, 
                  metrics='rmse',
                  early_stopping_rounds=100, 
                  stratified=False,
                  verbose_eval=100)


In [21]:
run_cv2(train_X, train_y)

[0]	train-rmse:1.74445+0.0100495	test-rmse:1.74424+0.0402173
[100]	train-rmse:1.42286+0.00461421	test-rmse:1.45328+0.0201143
[200]	train-rmse:1.33291+0.0054202	test-rmse:1.39011+0.0179483
[300]	train-rmse:1.28946+0.00541567	test-rmse:1.37149+0.0181799
[400]	train-rmse:1.26157+0.0063154	test-rmse:1.36654+0.0181094
[500]	train-rmse:1.24159+0.00795905	test-rmse:1.36419+0.0182677
[600]	train-rmse:1.22452+0.0091064	test-rmse:1.36274+0.0179345
[700]	train-rmse:1.20921+0.00948362	test-rmse:1.36174+0.0175072
[800]	train-rmse:1.19489+0.00983441	test-rmse:1.36096+0.0172712
[900]	train-rmse:1.18153+0.010173	test-rmse:1.36033+0.0168127
[1000]	train-rmse:1.16913+0.0103366	test-rmse:1.35969+0.016824
[1100]	train-rmse:1.15695+0.0104005	test-rmse:1.35903+0.0167686
[1200]	train-rmse:1.14551+0.0104635	test-rmse:1.35873+0.0168654
[1300]	train-rmse:1.13461+0.01023	test-rmse:1.35869+0.0171164
[1400]	train-rmse:1.1239+0.0097698	test-rmse:1.35856+0.0168756


In [32]:
def run_xgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        'booster': 'gbtree',
        'objective': 'reg:linear', 
        'learning_rate': 0.01,
        'max_depth': 3, 
        #'min_child_weight': 1,
        #'subsample': 0.1,
        #'colsample_bytree': 0.1,
        #'colsample_bylevel': 0.1,
        #'reg_lambda': 1,
        #'reg_alpha': 1,
        'base_score': np.mean(train_y)
    }
        
    xgtrain = xgb.DMatrix(train_X, label=train_y)
    xgval = xgb.DMatrix(val_X, label=val_y)
    
    watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
    
    bst = xgb.train(params, 
                    xgtrain, 
                    num_boost_round=10000, 
                    evals=watchlist,
                    early_stopping_rounds=50, 
                    verbose_eval=100)
    return bst

In [37]:
xgtest = xgb.DMatrix(test_X)
                    
folds = 3
kf = model_selection.KFold(n_splits=folds, shuffle=True, random_state=0)

sub_preds = np.zeros(test_X.shape[0])
oof_preds = np.zeros(train_X.shape[0])

for fold_no, (dev_index, val_index) in enumerate(kf.split(train_X)):
    dev_X, val_X = train_X.loc[dev_index,:], train_X.loc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    model = run_xgb(dev_X, dev_y, val_X, val_y, test_X)
    
    xgval = xgb.DMatrix(val_X, label=val_y)
    oof_preds[val_index] = model.predict(xgval, ntree_limit=model.best_ntree_limit )
    
    fold_rmse = np.sqrt(metrics.mean_squared_error(val_y, oof_preds[val_index]))
    print(f"fold {fold_no}: ", fold_rmse)    
    
    pred_test = model.predict(xgtest, ntree_limit=model.best_ntree_limit)    
    sub_preds += np.expm1(pred_test) / folds
    
    del xgval


[0]	train-rmse:1.73733	eval-rmse:1.75876
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:1.41444	eval-rmse:1.45853
[200]	train-rmse:1.32024	eval-rmse:1.39103
[300]	train-rmse:1.27329	eval-rmse:1.37099
[400]	train-rmse:1.24293	eval-rmse:1.3655
[500]	train-rmse:1.21952	eval-rmse:1.36274
[600]	train-rmse:1.20003	eval-rmse:1.36063
[700]	train-rmse:1.18253	eval-rmse:1.36002
Stopping. Best iteration:
[682]	train-rmse:1.18562	eval-rmse:1.35977

fold 0:  1.3597674593174112


ValueError: feature_names mismatch: ['20aa07010', 'dc5a8f1d8', '11d86fa6a', '4681de4fd', 'adf119b9a', 'b8a716ebf', '68b647452', '0d866c3d7', 'bd8f989f1', '22ed6dba3', '92b13ebba', '233c7c17c', '2cb4d123e', '87ffda550', '822e49b95', '316b978cd', 'd04e16aed', '11ad148bd', 'ea18d720e', '408d86ce9', '80b14398e', 'ecdef52b2', '2d6bd8275', 'dfdf4b580', 'feed9d437', '645b47cde', '7298ca1ef', '654dd8a3b', '8c94b6675', '0656586a4', 'b4ced4b7a', '0b8e10df6', '4e8196700', 'f115e74c0', '9ca0eee11', '472cd130b', 'a72e0bf30', '54b1c1bc0', 'ec1425047', 'bd550871c', '828b327a6', '21e0e6ae3', '64534cc93', '0f8d7b98e', '8367dfc36', '251d1aa17', '0a69cc2be', '963a49cdc', '587a5d8c3', '408d191b3', '42fdff3a0', '1e8801477', '29eddc376', 'ad207f7bb', '0564ff72c', '4e86dd8f3', '59dfc16da', '2f92a1a45', '63f968fa6', '4b6c549b1', 'a75d400b8', 'd318bea95', '35da68abb', '1a6d866d7', 'e059a8594', 'beb7f98fd', '99f466457', 'fe0dd1a15', 'f7faf2d9f', 'ffb34b926', 'bc2bf3bcd', '025dea3b3', 'e700276a2', '68a945b18', '

In [ ]:
sample = pd.read_csv('input/test.csv', usecols=['ID'])
sample["target"] = sub_preds
sample.to_csv("submissions/subm_20180629_xgb1.csv", index=False)